In [1]:
from brainlit.utils.session import NeuroglancerSession
from brainlit.utils.swc import graph_to_paths
import napari

/Users/alishakodibagkar/opt/anaconda3/envs/brainlit/lib/python3.8/site-packages/python_jsonschema_objects/__init__.py:50: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


# Downloading Brain data tutorial
We have prepared a 2 low-resolution brain volumes, as well as a single axon segment, at the below s3 urls (see `uploading_brains.ipynb`).
The method demonstrated below pulls a region of the volume around an annotated axon point set by the user.

## 1) Define Variables
- `mip` ranges from higher resolution (0) to lower resolution (1).
- `v_id` are vertex ids ranging from the soma (0) to the end of the axon (1649).
- `radius` is the radius to pull around the selected point, in voxels.

In [2]:
p = "s3://"
dir = p + "mouse-light-viz/precomputed_volumes/brain1"
dir_segments = p + "mouse-light-viz/precomputed_volumes/brain1_segments"
mip = 0
v_id = 300
radius = 75

## 2) Create a NeuroglancerSession instance and download the volume.

In [3]:
# get image and center point
ngl_sess = NeuroglancerSession(mip = mip, url = dir, url_segments=dir_segments)
img, bbox, vox = ngl_sess.pull_voxel(2, v_id, radius)
print(f"\n\nDownloaded volume is of shape {img.shape}, with total intensity {sum(sum(sum(img)))}.")

Downloading: 100%|██████████| 1/1 [00:00<00:00, 39.67it/s]
Downloading: 46it [00:01, 29.45it/s]                         



Downloaded volume is of shape (151, 151, 151), with total intensity 4946609.


## 3) Generate a graph from the segment data within the volume, and convert it to paths.

In [4]:
G_sub = ngl_sess.get_segments(2, bbox)
paths = graph_to_paths(G_sub)
print(f"Selected volume contains {G_sub.number_of_nodes()} nodes and {len(paths)} paths")

Downloading: 100%|██████████| 1/1 [00:00<00:00, 41.47it/s]


Selected volume contains 6 nodes and 2 paths


## 4) View the volume with paths overlaid via napari.

In [5]:
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(img)
    viewer.add_shapes(data=paths, shape_type='path', edge_width=0.1, edge_color='blue', opacity=0.1)
